# Installation
This section contains installation instruction and necessary imports.

In [ ]:
% cd ~
! git clone https://github.com/rigetticomputing/reference-qvm.git
% cd reference-qvm
! pwd
! ls
! pip install -r requirements.txt -e .
! pip install --upgrade pyquil

In [ ]:
# Standard library imports
import math
from math import pi
import cmath
import numpy as np
import random
import re

# PyQuil imports
import pyquil.quil as pq
from pyquil.quil import Program
from pyquil.quilbase import DefGate
from pyquil.gates import *

# Jupyter imporsts
from IPython.core.display import HTML, display

# license = """[Rigetti Forest] ##
# key: nmRPAVunQl19TtQz9eMd11iiIsArtUDTaEnsSV6u  ##
# user_id: 67c194ae-0ae4-4da8-8f3c-0d9f45359202""" ##
# open('.pyquil_config', 'w').write(license) ##

from referenceqvm.api import QVMConnection
#from pyquil.api.qvm import QVMConnection ##

# Syntax

## P1. The simplest program

We begin our journey with investigating this simple program.

First, we construct program containing two instructions `I 0`, and `X 1`. We don't care for their meaning now.

Then we instantiate connection (`QVMConnection()`) to our virtual quantum computer and execute program in two ways:

* first by explicitly measuring qbits (`run_and_measure`)
* second – by peeking the wavefunction (`wavefunction`).

This example shows what API is offered by the PyQuil library. Most of the functions we see here last time today, because we have some helper functions prepared that wrap all these numbers into nicer graphical representation.

You can also take a look on [official API documentation](https://pyquil.readthedocs.io/en/stable/quil.html).

In [ ]:
p = Program()
p.inst(I(0))
p.inst(X(1))

print('Program:', p, sep='\n')

print('Qubits', p.get_qubits())

qvm = QVMConnection()

print('Measurement', qvm.run_and_measure(p, [0, 1, 2]))

wavefunction = qvm.wavefunction(p)[0]

print('Wavefunction', wavefunction)
print('Amplitudes:', wavefunction.amplitudes, sep='\n')

## Helper functions

In [ ]:
# eg. 6 -> '011'  (3qbits)
def le_to_state(i, qbits_num):  
    fmt = '{{:0{}b}}'.format(qbits_num)
    return fmt.format(i)[::-1]

assert(le_to_state(6, 3) == '011')

# eg. '110' -> 6
def state_to_be(state):
    return int(state, 2)

assert(state_to_be('110') == 6)

# eg. 6 <-> 3  (3qbits)
def swap_endianness(i, qbits_num):
    return state_to_be(le_to_state(i, qbits_num))

assert(swap_endianness(6, 3) == 3)
assert(swap_endianness(3, 3) == 6)

# eg. 6 -> (0, 1, 1)  (3qbits)
def le_to_measurement(i, qbits_num):
    return tuple(map(int, le_to_state(i, qbits_num)))

assert(le_to_measurement(6, 3) == (0, 1, 1))

def format_program(program):    
    return re.sub(
        r'(DEFGATE [a-zA-Z0-9_]+:)\n((    .*\n)+)\n',
        r'\1<div class="matrix">\2</div>\n',
        str(program),
        flags=re.M | re.I)

def label_be(measurement):
    state_fmt = ''
    for qbit_id, bit in enumerate(measurement):
        state_fmt += '{}<sub>{}</sub>'.format(bit, qbit_id)
    return state_fmt

def format_html(amplitudes, measurements, program, label):
    qbits_num = int(math.log2(len(amplitudes)))

    num_measurements = len(measurements)
    stats = {}
    for measurement in measurements:
        measurement = tuple(measurement)
        stats[measurement] = stats.get(measurement, 0) + 1
  
    lines = []
    
    le_amplitudes = enumerate(amplitudes)
    be_amplitudes = {swap_endianness(i_le, qbits_num): amplitude for i_le, amplitude in le_amplitudes}

    for i_be, amplitude in sorted(be_amplitudes.items()):    
        i_le = swap_endianness(i_be, qbits_num)
        state = le_to_state(i_le, qbits_num)
                        
        mod = abs(amplitude)        
        hue = cmath.phase(amplitude)/2/math.pi*360
        
        measurement = le_to_measurement(i_le, qbits_num)
        measurements_count = stats.get(measurement, 0)
        
        lines.append('''<tr>
            <td class="be">{i_be}</td>
            <td class="s">|{state}⟩</td>
            <td class="a"><div class="wr"><div style="background: linear-gradient(hsl({hue}, 60%, 50%), hsl({hue}, 90%, 30%)); width: {width}%;"></div></div></td>
            <td class="ar {zero_class}">{areal:+.04f}</td>
            <td class="ai {zero_class}">{aimag:+.04f}ⅈ</td>
            <td class="m"><div class="wr"><div style="width: {mwidth}%;"</div></div></td>
            <td>{mcount}</td>
            </tr>'''.format(
                i_be=i_be,
                state=label(measurement),
                zero_class='zero' if mod==0 else '',
                areal=amplitude.real,
                aimag=amplitude.imag,
                hue=hue,
                width=mod*100,
                mcount=measurements_count,
                mwidth=measurements_count/num_measurements*100
            )
        )
      
    return '''<style>
        .amps{border:1px solid #000; border-collapse:collapse; font-family: monospace}
        .amps tr { border: 1px solid #111 }
        .amps tr th { text-align: center }
        .amps td { padding: 0.4em; vertical-align: middle; }
        .amps td.a { width: 10em; }
        .amps td .wr { border: 1px solid #555; height: 1em; width: 100%; box-sizing: border-box; background: #eee; }
        .amps td .wr div { height:100%; }
        .amps td.m { width: 5em; }
        .amps td.m .wr div { background: black; }
        .amps td.ar, .amps td.ai { font-family: Arial; font-size: 0.9em; }
        .amps td.ar.zero, .amps td.ai.zero { opacity: 0.5; }
        .amps td.s, .amps td.ai { border-right: 1px solid #333; }
        .amps td sub { opacity: 0.5; color: #024; font-size:0.5em; }
        .result { display: table-row; border-collapse: collapse;}
        .result pre, result div.right { display: table-cell }
        .result pre { background: linear-gradient(#fdfdb1, #dede83) !important;
            padding: 1em !important; padding-right: 5em !important; border: 1px solid black !important;
            border-right: 0px !important; min-width: 15em; max-width: 30em; }
        .result pre .matrix { font-size: .75em; line-height: 1em; }
    </style>''' + ''' 
    <div class="result">
    <pre>{program}</pre>
    <div class="right">
    <table class="amps">
    <tr>
    <th colspan="2">State</th>
    <th colspan="3">Amp (1 run)</th>
    <th colspan="2">Msr ({runs} runs)</th>
    </tr>'''.format(program=format_program(program),runs=num_measurements) +\
    '\n'.join(lines) +\
    '''</table></div></div>'''

def run_program(*instructions, **kwargs):  
    trials = kwargs.get('trials', 1)  
    quiet = kwargs.get('quiet', False)
    label = kwargs.get('label', label_be)
    
    program = pq.Program(*instructions)

    qvm = QVMConnection()
    measurements = qvm.run_and_measure(program, list(program.get_qubits()), trials=trials)
    amplitudes = qvm.wavefunction(program)[0].amplitudes

    if not quiet:
        display(HTML(format_html(amplitudes, measurements, program, label)))

    return qvm.classical_memory

## P2. Easy program

Now let's use and get familiar with our helper function `run_program`. As arguments you can write subsequent instructions. And that's all we should care about. Execution and displaying is handled by `run_program` function.


**Write any program.** You may use for instance following instructions:
`X(0)`, `Y(1)`, `Z(2)`.

Try and play around for a while – how these instructions affect the outcome? Don't be too detailed in experimentation – we are only familiarizing ourselfes with the syntax.

In [ ]:
run_program(
  X(0), ##  # Write instructions here
  Y(1), ##
  Z(2), ##
);

## P3. More about syntax

Now you have an idea how Quil instructions map to PyQuil functions.

For practise, please write the following program and execute it:
```
X 0
X 1
X 2
X 3
Y 1
Z 2
Z 3
```

In [ ]:
run_program(
    X(0), X(1), X(2), X(3), ## # Write instructions here
    Y(1), Z(2), Z(3) ##
);

Note that `Program` (and therefore `run_program` function) accepts:
* single instruction per argument,
* list of instructions,
* you can also nest `Program` in another `Program`.

Write the same program P3 below, using all three described methods.

Check if the produced output (program, states) is the same in both implementations.

In [ ]:
p = Program()
p.inst(Z(2), Z(3)) ### Append `Z 2` and `Z 3` instructions to `p` program.

run_program(
    [X(i) for i in range(4)], ## # Apply X gates
    Y(1),  ## # Apply Y gates
    p # `p` program is appended at the end
);

# Quantum computing postulates

These **are not** formal *postulates of quantum mechanics*. To skip math that is unnecessary at this point, I propose simplified, practical version:

0. Forget any intuition you have about classical computing. Don't think of boolean values, conditions, determinism, etc.
1. **System** of $N$ qbits has $2^N$ **base states**.
2. Each possible assignment of binary values ($\{0, 1\}$) to qbits is represented by one *base state*.
3. We can **label** *base states* in any way that is convenient, eg. $|0101⟩$ to indicate qbits' values in a certain order, or $|13⟩$, or even |🐭⟩.
4. Each base state has an associated **amplitude**.
5. We can see the current **state of a system** as a vector of $2^N$ numbers.
6. Amplitude is a [*complex* number](http://www.fluidstructureinteraction.com/numberbypaint/complex.html).
7. We interact with the system by applying **gates**, what usually effects in changing the amplitude (changing the state).
8. Each gate must be **reversible**.
9. If there is more than one state with non-zero amplitude, we say that the system is in **superposition** of base states.
10. We get information from the system by **measuring** a qbit. Measurement result is binary (either 0 or 1).
11. Measurement is **probabilistic**. Probability is related to the amplitude.
12. Measurement **collapses** the state.

If you have more formal knowledge on quantum computing, you can see where simplifications were made. Otherwise, these 13 points should be sufficient for the purpose of this workshop.

# Warmup

## P4. First nondeterministic

So far we used four types of gates `I`, `X`, `Y`, and `Z`. They are in fact identinty gate and three *Pauli* gates, respectively.

`X`, `Y`, `Z` perform an amplitude "swap" between pairs of states and/or change its phase.

There are also other gates that are able to move part of the amplitude (rotation gates):
```
RX(angle, qbit)
RY(angle, qbit)
RZ(angle, qbit)
```

Let's examine effect of these gates on pure state $|0⟩$ (the initial state of the quantum system).

Once you manage to get nonzero amplitude in both $|0⟩$ and $|1⟩$ states, add `trials=100` parameter to the `run_program` call and watch the count of measurements.

What is the relationship between amplitude and number of measurements?

In [ ]:
run_program(
    RX(0.6, 0), ## # Write instruction here
##
    trials=100 ##
);

## P5. Hadamard gate

Now let's get familiar with *Hadamard gate* `H` (\[adamaʁ\]). It is another gate.

One of its properties is that it transforms $|0⟩$ state to two equal amplitudes ($\frac{1}{\sqrt2}|0⟩ + \frac{1}{\sqrt2}|1⟩$ – in fact).

Write the simplest program to examine this behavior:
```
H 0
```

Use also `trials=` parameter what is the proportion between number of measurements for each state.

In [ ]:
run_program(
    H(0), ## # Write instruction here
##
    trials=100 ##
);

Let us see now how Hadamard gate affects state $|1⟩$.

How is it different from previous result? Is information about previous state ($|0⟩$ or $|1⟩$) lost?

In [ ]:
run_program(
    X(0), ## # Prepare |1> state
    H(0), ## # Apply Hadamard
##
    trials=100 ##
);

Assume we want to undo this operation. One interesting property is that $HH = I$ (we can write also $H^2=I$, and that indicates there is some math behind it). This also works for the gates $X$, $Y$, and $Z$.

Add some instruction(s) to the previous program to go back to the state $|1⟩$. Do some experiments.

In [ ]:
run_program(
    X(0), ## # (Copy above program here)
    H(0), ##
##
    H(0), ## # Undo Hadamard gate by adding new instruction.
##
    trials=100 ##
);

## P6. Measurement – classical registry

The information we get from real quantum computer has form of classical bits. We cannot peek the state directly.

Therefore only by measuring qbits we can fetch any results of computation. To do it in a proper way, Quil introduces a classic registry – an array of bits which can be read and written during the execution.

Instruction for measuring is defined as `MEASURE(qbit_id, creg_pos)` and performs a measurement operation for a single qbit (numbered `qbit_id`), and stores the result in the classical registry at position `creg_pos`.

Examine the following program. Why the amplitude is now concentrated only in one state? Which one?

Does amplitude and result of measurement differ? Why is that?

In [ ]:
creg = run_program(
  H(0),
  MEASURE(0, 0)
)

print('Random result is:', creg[0])

As an excercise, write a quantum-based program, that can generates random number $x \in \{0, 1, 2, 3, 4, 5, 6, 7\}$ with equal probability.

In [ ]:
creg = run_program(
    H(0), H(1), H(2), ## # Write your program here
    MEASURE(0, 0), ##
    MEASURE(1, 1), ##
    MEASURE(2, 2) ##
)

number = creg[0]*4 + creg[1]*2 + creg[2] ##number = ???
print('Random number =', number)

## P7. „Hello world”

Now we will prepare very interesting quantum state, called *entanglement*.

A new gate is introduced, `CNOT(x, y)` – which operates on two qbits. You can imagine it works like:

```
if x:
  y := not y
```

This is why it is called *controlled not*. And by the way this is not the most precise definition.

Examine how it evolves state $|00⟩$ and $|10⟩$

In [ ]:
run_program(
    I(0), I(1), # No-operation, now we have state |00>
    CNOT(0, 1), ## # Add CNOT instruction
#    MEASURE(0, 0),##    
#    trials=100 ##
);

run_program(
    X(0), I(1), # X gate, now we have state |10>
    CNOT(0, 1), ## # Add CNOT instruction
#    MEASURE(0, 0),##
#    trials=100 ##
);

How does `CNOT` work if a system is in superposition? You can make calculation for each basic state and then merge (add) results.

Can you prepare system to be in state $|00⟩$ + $|11⟩$ (with equal amplitudes of $\frac{1}{\sqrt2}$)?

Once you've done it, think of its physical interpretation. This result is fascinating!

In [ ]:
run_program(
    H(0), ## # Write program here
    CNOT(0, 1), ##
#    MEASURE(0, 0),##
#    trials=100 ##
);

# Simple
Now we know how to use the PyQuil library and our helper functions. Let's start getting some real quantum fun!

## P8. Hybrid programming

Prepare a single qubit in a state that all the amplitude is concentrated in state $|1⟩$. In the beginning we assume that you have qubit in random state (here emulated by 3 random rotations).

`Program` object has method `if_then(classical_reg, if_program, else_program=None)`. You'd like to use it.

In [ ]:
prepare_random_qubit = Program().inst(
    RX(random.random()*pi*4, 0),
    RY(random.random()*pi*4, 0),
    RZ(random.random()*pi*4, 0)
)

p = Program()
p.inst(prepare_random_qubit)

p.measure(0, 0)  ### Write something here?
p.if_then(0, [], X(0)) ##p.if_then(???, ???, ???)
# Write something here?

run_program(p);

Can you prepare this qubit in a state where amplitude is equal exactly to $+1$ (i.e. no imaginary part)?

In [ ]:
prepare_random_qubit = Program().inst(
    RX(random.random()*pi*4, 0),
    RY(random.random()*pi*4, 0),
    RZ(random.random()*pi*4, 0)
)

p = Program()
p.inst(prepare_random_qubit)

p.measure(0, 0)  ### Write something here?
p.if_then(0, [], X(0)) ##p.if_then(???, ???, ???)
# Write something here?

run_program(p);

## P9. Custom gate

Before continuing we need to get more formal knowledge on how gates work.

They are *linear operators* and can be represented as complex-valued square unitary matrices of size $M \times M$ where $M = 2^N$, where $N$ is the number of qbits it affects.

Then, analyze this program.

In [ ]:
run_program(
  H(0),
  I(1)
);

State evolution obeys the following rule:

$|\psi'⟩ = M|\psi⟩$

where $|\psi⟩$ is the previous state (vector of amplitudes), $|\psi'⟩$ is the next state, and $M$ is an unitary matrix.



Our current state is $\frac{1}{\sqrt2}|00⟩ + \frac{1}{\sqrt2}|10⟩$, what can be represented as a vector:

$|\psi⟩ = \begin{bmatrix}\frac{1}{\sqrt2} \\ 0 \\ \frac{1}{\sqrt2} \\ 0\end{bmatrix}$

We want to move amplitude from state $|10⟩$ to $|01⟩$, in order to reach state $\frac{1}{\sqrt2}|00⟩ + \frac{1}{\sqrt2}|01⟩$.

We have the ability to apply custom matrix $M$. Edit gate definition matrix so it realizes the amplitude swap between $|10⟩$ and $|01⟩$. Current implementation is an identity matrix (does nothing).

In [ ]:
DEF_MY = DefGate('MY', np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0], ## [0, 1, 0, 0],
    [0, 1, 0, 0], ## [0, 0, 1, 0],
    [0, 0, 0, 1]
], dtype=complex))

run_program(   
    DEF_MY, # Gate definition
    
    H(0),
    I(1),
    ('MY', 0, 1), # Applying gate on qbits 0 and 1.
#    trials=100 ##
);

In this case, we were able to acheive the same result using two CNOT-s.

In [ ]:
run_program(
    H(0),
    I(1),
    
    CNOT(0, 1),
    CNOT(1, 0)
);

In general, we change amplitude between $|10⟩$ and $|01⟩$, and leave the state unchanged for $|00⟩$ and $|11⟩$.

You can realize that it swaps these two qbits. This is a `SWAP` instruction. Guess its arguments and check if it results in the same state.

In [ ]:
run_program(
    H(0),
    I(1),

    SWAP(0, 1) ## # Swap instruction?
);

## P10. Our custm functions cont. – Is prime?

Based on our previous excercies we create simple helper function to write amplitude swaps (between two given states) into a matrix in a convenient way.

In [ ]:
def swap_states_be(matrix, i, j):
    tmp = matrix[i].copy()
    matrix[i] = matrix[j]
    matrix[j] = tmp

Common pattern for defining quantum functions/gates is transformation of pair $(X, Y) \rightarrow (X, Y \oplus f(X))$. This way, the operation is easily revertible, what is a requirement for any quantum gate.

Let us define a gate that implements a function "returning" 1 if given number (X) is prime and 0 if it is not.

It means that it must contain swap between $|X, Y⟩$ and $|X, \neg Y⟩$ if and only if $X$ is prime. Let us use values of $X \in \{0, 1, 2, 3\}$, and $Y$ for representing *false* and *true*.

Note the unusual state notation. We must create bijection between previously used binary notation for states and our current needs. The natural choice is to map $X$ to binary (i.e. 0 $\rightarrow$ 00, 1 $\rightarrow$ 01, 2 $\rightarrow$ 10, 3 $\rightarrow$ 11) and encode it on two qubits (0. and 1.). And Y can be naturally encoded on the remaining one (2. qbit). This is one of many possible choices, but the most convenient one.

For instance then we can interpret:
* $|101⟩$ as $|2, $*true*$⟩$
* $|110⟩$ as $|3, $*false*$⟩$

The following table might help you understand what is the desired effect:

| Before  | X | Y     |Prime?| After   |        
| --------|:-:|:-----:|:----:| ------- |
| $|000⟩$ | 0 | false |      | $|000⟩$ | 
| $|001⟩$ | 0 | true  |      | $|001⟩$ |
| $|010⟩$ | 1 | false |      | $|010⟩$ | 
| $|011⟩$ | 1 | true  |      | $|011⟩$ |
| $|100⟩$ | 2 | false | yes! | $|101⟩$ | 
| $|101⟩$ | 2 | true  | yes! | $|100⟩$ |


Your task is to write appropriate matrix for applying our „ISPRIME” operation.


In [ ]:
is_prime_mat = np.eye(2**3, dtype=np.complex) # Creates identity matrix

swap_states_be(is_prime_mat, 2<<1, (2<<1) + 1) ##swap_states_be(is_prime_mat, ???, ???)
swap_states_be(is_prime_mat, 3<<1, (3<<1) + 1) ##swap_states_be(is_prime_mat, ???, ???)

DEF_ISPRIME = DefGate('ISPRIME', is_prime_mat)
ISPRIME = DEF_ISPRIME.get_constructor()

def custom_label(measurement):
    return str(int(''.join(map(str,measurement[:-1])), 2)) + ['-', '+'][measurement[-1]] + ' @ ' + label_be(measurement)

for test in [0, 1, 2, 3]:
    
    display(HTML('<h2>Is {} prime?</h2>'.format(test)))
    
    prepare_number = Program()
    if test & 1:
        prepare_number.inst(X(1))
    if test & 2:
        prepare_number.inst(X(0))

    creg = run_program(
        DEF_ISPRIME,
        I(0), I(1),
        I(2),   

        prepare_number,

        ISPRIME(0, 1, 2),
        MEASURE(2, 0),
        
        label=custom_label
    )

    print('It seems that {} is {}'.format(test, 'prime' if creg[0] else 'not prime'))

Another helper function that wraps creation of this kind of gates.

`defgate_set(name, qbits_num, content)` accepts list of integers from range $[0; 2^{\texttt{qbits_num}})$ for which we want to perform swap.

In [ ]:
def defgate_set(name, qbits_num, content):
    mat = np.eye(2**(qbits_num + 1), dtype=np.complex)
    
    for number in set(content):
        if type(number) is not int or number < 0 or number >= 2**qbits_num:
            raise ValueError('Cannot represent {} in {}-qbit gate'.format(number, qbits_num))
        swap_states_be(mat, number<<1, (number<<1) + 1)
        
    defgate_instr = DefGate(name, mat)
    return defgate_instr, defgate_instr.get_constructor()

Define gates:
* `ISPRIME2` (this one we defined above, on 2 qbits);
* `ISPRIME3`, `ISINTEGER`, `ISPOSITIVE`, `ISPERFECT`, `ISNEGATIVE` (on 3 qbits).

Use `defgate_set` function. Test each created function.

We'll need them later.

In [ ]:
DEF_ISPRIME2, ISPRIME2 = defgate_set('ISPRIME2', 2, [2, 3])
DEF_ISPRIME3, ISPRIME3 = defgate_set('ISPRIME3', 3, [2, 3, 5, 7]) ##DEF_ISPRIME3, ISPRIME3 = defgate_set('ISPRIME3', 3, [???])
DEF_ISINTEGER, ISINTEGER = defgate_set('ISINTEGER', 3, [0, 1, 2, 3, 4, 5, 6, 7]) ##DEF_ISINTEGER, ISINTEGER = ???
DEF_ISPOSITIVE, ISPOSITIVE = defgate_set('ISPOSITIVE', 3, [1, 2, 3, 4, 5, 6, 7]) ##DEF_ISPOSITIVE, ISPOSITIVE = ???
DEF_ISPERFECT, ISPERFECT = defgate_set('ISPERFECT', 3, [6]) ##DEF_ISPERFECT, ISPERFECT = ???
DEF_ISNEGATIVE, ISNEGATIVE = defgate_set('ISNEGATIVE', 3, []) ##DEF_ISNEGATIVE, ISNEGATIVE = ???
DEF_ISODD, ISODD = defgate_set('ISODD', 3, [1,3,5,7]) ##DEF_ISODD, ISODD = ???

for test in [0, 1, 2, 3, 4, 5, 6, 7]:
        
    prepare_number = Program()
    if test & 1:
        prepare_number.inst(X(2))
    if test & 2:
        prepare_number.inst(X(1))
    if test & 4:
        prepare_number.inst(X(0))
        
    creg = run_program(
        DEF_ISPRIME3, # Gate definition
        I(0),
        prepare_number,

        ISPRIME3(0, 1, 2, 3), # Gate usage
        MEASURE(3, 0),
        
        quiet=True
    )
    print('For number {}, the answer is {}'.format(test, creg[0]))

**Important notice**

At this point it might seem to you that defining functions in quantum environment comes down to defining one big permutation matrix. We do it here this way to simplify this workshop.

In real cases classical boolean function can be implemented as a sequence of simple quantum gates (see [this work](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.113.8040&rep=rep1&type=pdf)). But in the end, their product is equal to the permutation matrix we create here.

## P11. All at once

Now do the same, but check all numbers at once (prepare state – don't do measurement).

In [ ]:
run_program(
    DEF_ISPRIME3,
    I(0), ## # Write program here
##
    H(0), ##
    H(1), ##
    H(2), ##
##
    ISPRIME3(0, 1, 2, 3) ##
);

Now the task is to select prime numbers only.

Measuring the state can be preceived as a tool for zeroing the amplitude in places not matching our measurement.  We can't control the measurement result, however. It is driven by probability.

The expected state is $C(|2, $*true*$⟩ + |3, $*true*$⟩ + |5, $*true*$⟩ + |7, $*true*$⟩)$ for some constant $C$.

In [ ]:
while True:    
    creg = run_program(
        DEF_ISPRIME3,
        I(0), ##  # Write program here
##
        H(0), ##
        H(1), ##
        H(2), ##
##     
        ISPRIME3(0, 1, 2, 3), ##
##
        MEASURE(3, 0), ##  MEASURE(???, ???)
    )
    
    if creg[0] == False:
        print('This run failed. Running again')
    else:
        break

How could we implement the above program using only Quil instructions? Hint: `while_do(classical_reg, q_program)`.

# Quantum magic begins

## P12. Quantum magic begins

Let's begin with the following program:
```
X 1
H 1
H 0

CNOT 0 1

H 0
```

What is the state of 0-th qbit?

In [ ]:
run_program(
  X(1), ## # Write program here
  H(1), ##
##
  H(0), ##   
  CNOT(0, 1), ##
## 
  H(0), ##
);

Then run another program, which does not contain `CNOT` instruction.
What is the state now?

Explain?

In [ ]:
run_program(
  X(1), ## # Write program here
  H(1), ##
##
  H(0), ##
##
  H(0) ##
);

## P13. Deutsch algorithm
We discovered our first algorithm that tests functions (called *oracles*).

Requirements for "oracle" gate:
* $(X, Y) \rightarrow (X, Y \oplus f(X))$,
* $f(X)$ either balanced or constant in its domain (here: $\{0, 1\}$).

Code:
```
X 1
H 0
H 1
ORACLE 0 1
H 0
MEASURE 0 [0]
```

Measured value tells if the function is constant or balanced.

In [ ]:
x_reg = 0
y_reg = 1

oracles = {
    'balanced1': [
        CNOT(x_reg, y_reg) # An example of balanced function
    ],
    'balanced2': [
        CNOT(x_reg, y_reg), ##  # Write another balanced function
        X(y_reg), ##
    ],
    'constant': [
        X(y_reg) ##  # Write constant function
    ],
}

for oracle_name, oracle_code in oracles.items():
    display(HTML('<h2>{}</h2>'.format(oracle_name)))

    creg = run_program(
        X(1),
        H(0),
        H(1),

        oracle_code,

        H(0),
        MEASURE(0, 0)
    )

    if creg[0] == 0:
        print('This function is constant')
    else:
        print('This function is balanced')

Note that we called "Oracle" function only once, and that determined the result (whether balanced or constant).

How many calls would you need to solve this problem on classical computer?

## P14. Deutsch-Jozsa algorithm – first quantum speedup

Now we want to write generic version of Deutsch algorithm...

In [ ]:
function_qbits_num = 3 # Number of input qbits (x registry)

x_reg = tuple(range(0, function_qbits_num)) # 0, 1, 2

y_reg = function_qbits_num # 3 

oracles = {
    'balanced1': [
        CNOT(x_reg[0], y_reg), # An example of balanced function
        CNOT(x_reg[1], y_reg)
    ],
    'balanced2': [
        CNOT(x_reg[0], y_reg), ##  # Write another example
        X(y_reg), ##
    ],

    'constant1': [
        X(y_reg) ##  # Write example function
    ],

    'constant2': [
        I(y_reg) ##  # Write another example
    ], 

    'isprime3': [ # We can also test our gates we defined earlier.
        DEF_ISPRIME3, 
        ISPRIME3(*x_reg, y_reg)
    ], 
    
    'isinteger': [ ## # Test another gates (like ISINTEGER, ISNEGATIVE)
        DEF_ISINTEGER, ##
        ISINTEGER(*x_reg, y_reg) ##
    ], ##
    'isnegative': [ ##
        DEF_ISNEGATIVE, ##
        ISNEGATIVE(*x_reg, y_reg) ##
    ], ##
    
    'isperfect': [ ## # What happens when we put neither balanced nor constant (like ISPERFECT) gate
        DEF_ISPERFECT, ##
        ISPERFECT(*x_reg, y_reg) ##
    ], ##
}

def custom_label(state):
    return state

for oracle_name, oracle_code in oracles.items():
  
    display(HTML('<h2>{}</h2>'.format(oracle_name)))

    creg = run_program(
        X(y_reg), ##  # Write complete DJ algorithm
##
        [H(i) for i in range(function_qbits_num+1)],##
##
        oracle_code,##
##
##
        [H(i) for i in range(0, function_qbits_num)],##
##
        [MEASURE(i, i) for i in range(0, function_qbits_num)],##
      ##  
       ## 
        trials=10,##
        label=custom_label
    )
  
    if (creg[0:function_qbits_num] == 0).all(): ## if ???:
        print('This function is constant')
    else:
        print('This function is balanced')

# What next?

Please, fill the [feedback form](https://goo.gl/forms/GZJD4fb8y5UlTGAT2).

## Want more?
* You can [read official tutorial on PyQuil](https://pyquil.readthedocs.io/en/stable/index.html)
* You can apply for acces to quantum simulator/computer in the cloud [on Rigetti Forst website](https://www.rigetti.com/forest)
* You can take a look at [the Grove library](https://github.com/rigetticomputing/grove/tree/master/grove) – containing a number of quantum algorithms implemented in PyQuil
* You can try another [introduction to PyQuil](http://dkopczyk.quantee.co.uk/high-level-quantum-computing/) 
